# Paper Trade Logic

In [1]:
import pandas as pd

balance = 10000.00
totalNumberOfShares = 0
unRealisedProfit = 0.00
brokerageFee = 20.00
tax = 0.02
transactionDict = dict()
totalTaxesPaid = 0.00
transaction_df = pd.DataFrame(columns=["Type", "Shares", "PricePerShare", "TotalAmount", "Balance", "UnrealisedProfit"])


def calculate_unrealised_profit(current_price):
    return current_price * totalNumberOfShares


def trade(tradeType, tradeNumberOfShares, tradePricePerShare, balance):
    global totalNumberOfShares
    global unRealisedProfit
    global totalTaxesPaid
    earnedAmount = 0.00

    if tradeType == 1:
        requiredAmountForBuy = (tradeNumberOfShares * tradePricePerShare
                                + ((tradeNumberOfShares * tradePricePerShare) * tax))
        if requiredAmountForBuy < balance:
            balance -= requiredAmountForBuy
            totalNumberOfShares += tradeNumberOfShares
            totalTaxesPaid += (tradeNumberOfShares * tradePricePerShare) * tax
            unRealisedProfit = calculate_unrealised_profit(tradePricePerShare)
            earnedAmount = 0.00
        else:
            print("Balance is not enough for the trade")

    elif tradeType == 0:
        if tradeNumberOfShares <= totalNumberOfShares:
            earnedAmount = (tradeNumberOfShares * tradePricePerShare
                            - ((tradeNumberOfShares * tradePricePerShare) * tax)
                            - brokerageFee)
            totalNumberOfShares -= tradeNumberOfShares
            totalTaxesPaid += (tradeNumberOfShares * tradePricePerShare) * tax + brokerageFee
            balance += earnedAmount
            unRealisedProfit = calculate_unrealised_profit(tradePricePerShare)
        else:
            print("Not enough shares to sell.")

    transactionDict[len(transactionDict) + 1] = {
        "Type": "Buy" if tradeType == 1 else "Sell",
        "Shares": tradeNumberOfShares,
        "PricePerShare": tradePricePerShare,
        "TotalAmount": requiredAmountForBuy if tradeType == 1 else earnedAmount,
        "Balance": balance,
        "UnrealisedProfit": unRealisedProfit
    }

    transaction_df.loc[len(transaction_df)] = [
        "Buy" if tradeType == 1 else "Sell",
        tradeNumberOfShares,
        tradePricePerShare,
        requiredAmountForBuy if tradeType == 1 else earnedAmount,
        balance,
        unRealisedProfit
    ]

    return balance, totalNumberOfShares, unRealisedProfit, transaction_df








def print_transaction_history():
    for idx, transaction in transactionDict.items():
        print(f"Transaction {idx}:")
        print("Type:", transaction["Type"])
        print("Shares:", transaction["Shares"])
        print("Price Per Share:", transaction["PricePerShare"])
        print("Total Amount:", transaction["TotalAmount"])
        print("Balance:", transaction["Balance"])
        print("Unrealised Profit:", transaction["UnrealisedProfit"])
        print()

# Main trading loop
while True:
    print("\nCurrent Balance:", balance)
    print("Total Number of Shares:", totalNumberOfShares)
    print("Unrealised Profit:", unRealisedProfit)

    choice = input("\nEnter 'b' to buy shares, 's' to sell shares, or 'd' to square off and exit: ")

    if choice.lower() == "d":
        break

    if choice.lower() == "b" or choice.lower() == "s":
        tradeType = 1 if choice.lower() == "b" else 0

        if tradeType == 0 and totalNumberOfShares == 0:
            print("You don't have any shares to sell.")
            continue

        tradeNumberOfShares = int(input("Enter the number of shares: "))
        pricePerShare = float(input("Enter the price per share: "))
        balance, totalNumberOfShares, unRealisedProfit, transaction_df = trade(tradeType, tradeNumberOfShares,
                                                                               pricePerShare, balance)  # Corrected line
        print("Transaction successful!")

print("\nFinal Summary:")
print("Updated Balance:", balance)
print("Total Number of Shares:", totalNumberOfShares)
print("Unrealised Profit:", unRealisedProfit)
print('Total Tax Paid: ', totalTaxesPaid)

if unRealisedProfit > 0:
    print("\nProfit Square Off: You have made a profit of", unRealisedProfit)
elif unRealisedProfit < 0:
    print("\nLoss Square Off: You have incurred a loss of", -unRealisedProfit)

print("\nTransaction History:")
print_transaction_history()


Current Balance: 10000.0
Total Number of Shares: 0
Unrealised Profit: 0.0

Enter 'b' to buy shares, 's' to sell shares, or 'd' to square off and exit: d

Final Summary:
Updated Balance: 10000.0
Total Number of Shares: 0
Unrealised Profit: 0.0
Total Tax Paid:  0.0

Transaction History:


In [2]:
transaction_df

,Type,Shares,PricePerShare,TotalAmount,Balance,UnrealisedProfit


# Calculator Logic

In [3]:
def calculate_equity_delivery(quantity, buy_price, sell_price):
    brokerage_rate = 0.0003  # 0.03% brokerage
    max_brokerage = 20  # Maximum brokerage amount
    stt_buy_rate = 0.001  # 0.1% STT on buy side
    stt_sell_rate = 0.001  # 0.1% STT on sell side
    exchange_fee_nse = 0.0000325  # 0.00325% NSE transaction charge
    exchange_fee_bse = 0.0000375  # 0.00375% BSE transaction charge
    gst_rate = 0.18  # 18% GST
    sebi_charge = 10  # ₹10 per crore SEBI charges
    stamp_duty_buy_rate = 0.00015  # 0.015% stamp duty on buy side
    stamp_duty_sell_rate = 0.00015  # 0.015% stamp duty on sell side

    buy_amount = quantity * buy_price
    sell_amount = quantity * sell_price
    turnover = buy_amount + sell_amount

    brokerage = min(max_brokerage, max(buy_amount * brokerage_rate, 20))
    stt_buy = buy_amount * stt_buy_rate
    stt_sell = sell_amount * stt_sell_rate
    exchange_transaction_charge_nse = sell_amount * exchange_fee_nse
    exchange_transaction_charge_bse = sell_amount * exchange_fee_bse
    gst = (brokerage + sebi_charge + exchange_transaction_charge_nse + exchange_transaction_charge_bse) * gst_rate
    sebi_charges = turnover * (sebi_charge / 10000000)  # Converting crore to actual value
    stamp_duty_buy = buy_amount * stamp_duty_buy_rate
    stamp_duty_sell = sell_amount * stamp_duty_sell_rate

    total_tax_and_charges = brokerage + stt_buy + stt_sell + exchange_transaction_charge_nse + exchange_transaction_charge_bse + gst + sebi_charges + stamp_duty_buy + stamp_duty_sell
    net_profit = sell_amount - buy_amount - total_tax_and_charges
    points_to_breakeven = abs(net_profit) / quantity

    return {
        
#         turnover, brokerage, stt_buy, stt_sell, exchange_transaction_charge_nse, 
#             exchange_transaction_charge_bse, gst,sebi_charges, stamp_duty_buy, stamp_duty_sell, total_tax_and_charges,
#             points_to_breakeven, net_profit
            
        "Turnover": turnover,
        "Brokerage": brokerage,
        "STT Buy": stt_buy,
        "STT Sell": stt_sell,
        "Exchange Txn Charge NSE": exchange_transaction_charge_nse,
        "Exchange Txn Charge BSE": exchange_transaction_charge_bse,
        "GST": gst,
        "SEBI Charges": sebi_charges,
        "Stamp Duty Buy": stamp_duty_buy,
        "Stamp Duty Sell": stamp_duty_sell,
        "Total Tax and Charges": total_tax_and_charges,
        "Points to Breakeven": points_to_breakeven,
        "Net Profit": net_profit
    }

if __name__ == "__main__":
    quantity = 400
    buy_price = 1000
    sell_price = 0

    result = calculate_equity_delivery(quantity, buy_price, sell_price)

    for key, value in result.items():
        print("{}: {:.2f}".format(key, value))


Turnover: 400000.00
Brokerage: 20.00
STT Buy: 400.00
STT Sell: 0.00
Exchange Txn Charge NSE: 0.00
Exchange Txn Charge BSE: 0.00
GST: 5.40
SEBI Charges: 0.40
Stamp Duty Buy: 60.00
Stamp Duty Sell: 0.00
Total Tax and Charges: 485.80
Points to Breakeven: 1001.21
Net Profit: -400485.80


In [4]:
result

{'Turnover': 400000,
 'Brokerage': 20,
 'STT Buy': 400.0,
 'STT Sell': 0.0,
 'Exchange Txn Charge NSE': 0.0,
 'Exchange Txn Charge BSE': 0.0,
 'GST': 5.3999999999999995,
 'SEBI Charges': 0.39999999999999997,
 'Stamp Duty Buy': 59.99999999999999,
 'Stamp Duty Sell': 0.0,
 'Total Tax and Charges': 485.79999999999995,
 'Points to Breakeven': 1001.2144999999999,
 'Net Profit': -400485.8}

#  Now Add  Calulator logic to Trading Account

In [9]:
import pandas as pd

balance = 10000.00
totalNumberOfShares = 0
unRealisedProfit = 0.00
brokerageFee = 20.00
tax = 0.02
transactionDict = dict()
totalTaxesPaid = 0.00
transaction_df = pd.DataFrame(columns=["Type", "Shares", "PricePerShare", "TotalAmount", "Balance", "UnrealisedProfit"])


def calculate_unrealised_profit(current_price):
    return current_price * totalNumberOfShares


def trade(tradeType, tradeNumberOfShares, tradePricePerShare, balance):
    global totalNumberOfShares
    global unRealisedProfit
    earnedAmount = 0.00

    if tradeType == 1:
        requiredAmountForBuy = (tradeNumberOfShares * tradePricePerShare)
        if requiredAmountForBuy < balance:
            balance -= requiredAmountForBuy
            totalNumberOfShares += tradeNumberOfShares
            earnedAmount = 0.00
        else:
            print("Balance is not enough for the trade")

    elif tradeType == 0:
        if tradeNumberOfShares <= totalNumberOfShares:
            earnedAmount = (tradeNumberOfShares * tradePricePerShare)
            totalNumberOfShares -= tradeNumberOfShares
            balance += earnedAmount
        else:
            print("Not enough shares to sell.")

    transactionDict[len(transactionDict) + 1] = {
        "Type": "Buy" if tradeType == 1 else "Sell",
        "Shares": tradeNumberOfShares,
        "PricePerShare": tradePricePerShare,
        "TotalAmount": requiredAmountForBuy if tradeType == 1 else earnedAmount,
        "Balance": balance,
        "UnrealisedProfit": calculate_unrealised_profit(tradePricePerShare)
    }

    transaction_df.loc[len(transaction_df)] = [
        "Buy" if tradeType == 1 else "Sell",
        tradeNumberOfShares,
        tradePricePerShare,
        requiredAmountForBuy if tradeType == 1 else earnedAmount,
        balance,
        calculate_unrealised_profit(tradePricePerShare)
    ]

    return balance, totalNumberOfShares, unRealisedProfit, transaction_df


def print_transaction_history():
    for idx, transaction in transactionDict.items():
        print(f"Transaction {idx}:")
        print("Type:", transaction["Type"])
        print("Shares:", transaction["Shares"])
        print("Price Per Share:", transaction["PricePerShare"])
        print("Total Amount:", transaction["TotalAmount"])
        print("Balance:", transaction["Balance"])
        print("Unrealised Profit:", transaction["UnrealisedProfit"])
        print()

# Main trading loop
while True:
    print("\nCurrent Balance:", balance)
    print("Total Number of Shares:", totalNumberOfShares)
    print("Unrealised Profit:", unRealisedProfit)

    choice = input("\nEnter 'b' to buy shares, 's' to sell shares, or 'd' to square off and exit: ")

    if choice.lower() == "d":
        break

    if choice.lower() == "b" or choice.lower() == "s":
        tradeType = 1 if choice.lower() == "b" else 0

        if tradeType == 0 and totalNumberOfShares == 0:
            print("You don't have any shares to sell.")
            continue

        tradeNumberOfShares = int(input("Enter the number of shares: "))
        pricePerShare = float(input("Enter the price per share: "))
        balance, totalNumberOfShares, unRealisedProfit, transaction_df = trade(tradeType, tradeNumberOfShares,
                                                                               pricePerShare, balance)  # Corrected line
        print("Transaction successful!")

# Calculate total tax and charges and net profit after trading loop
total_tax_and_charges = transaction_df["TotalAmount"].sum() - balance
net_profit = unRealisedProfit - total_tax_and_charges

print("\nFinal Summary:")
print("Updated Balance:", balance)
print("Total Number of Shares:", totalNumberOfShares)
print('Total Tax and Charges:', total_tax_and_charges)
print("Net Profit:", net_profit)

print("\nTransaction History:")
print_transaction_history()



Current Balance: 10000.0
Total Number of Shares: 0
Unrealised Profit: 0.0

Enter 'b' to buy shares, 's' to sell shares, or 'd' to square off and exit: b
Enter the number of shares: 10
Enter the price per share: 10
Transaction successful!

Current Balance: 9900.0
Total Number of Shares: 10
Unrealised Profit: 0.0

Enter 'b' to buy shares, 's' to sell shares, or 'd' to square off and exit: s
Enter the number of shares: 10
Enter the price per share: 15
Transaction successful!

Current Balance: 10050.0
Total Number of Shares: 0
Unrealised Profit: 0.0

Enter 'b' to buy shares, 's' to sell shares, or 'd' to square off and exit: d

Final Summary:
Updated Balance: 10050.0
Total Number of Shares: 0
Total Tax and Charges: -9800.0
Net Profit: 9800.0

Transaction History:
Transaction 1:
Type: Buy
Shares: 10
Price Per Share: 10.0
Total Amount: 100.0
Balance: 9900.0
Unrealised Profit: 100.0

Transaction 2:
Type: Sell
Shares: 10
Price Per Share: 15.0
Total Amount: 150.0
Balance: 10050.0
Unrealised Pr

(10050.0,
 0,
 0.0,
    Type  Shares  PricePerShare  TotalAmount  Balance  UnrealisedProfit
 0   Buy      10           10.0        100.0   9900.0             100.0
 1  Sell      10           15.0        150.0  10050.0               0.0
 2  Sell      10           15.0          0.0  10050.0               0.0)